## Mode of Simulation 

Three type of simulation 

<strong>1. Profit maximization</strong> 

<strong>2. Solo Rides</strong>

<strong>3. Pooled Rides</strong>

## Load ExMAS and MaaSSim


In [269]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
import logging
import matplotlib.pyplot as plt

import pandas as pd
import ExMAS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Delft, Netherlands

In [270]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 1 # 6 minutes hour of simulation
params.nP = 50 # reuqests (and passengers)
params.nV = 20 # vehicles


## Parameters for ExMAS

In [271]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.0005
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True



# params.kpi = 1 (Profit Maximazation)
# params.kpi = 2 (Solo Rides)
# params.kpi = 3 (Pooled Rides)


In [272]:
params.kpi = 2

### Profit Mazimization - Begin 

In [273]:
#inData = ExMAS.main(inData, params.shareability) # create shareability graph (ExMAS) 
inData = ExMAS.main(inData, params.shareability, plot = False)

11-07-23 23:48:25-INFO-Initializing pairwise trip shareability between 50 and 50 trips.
11-07-23 23:48:26-INFO-creating combinations
11-07-23 23:48:26-INFO-2450	 nR*(nR-1)
11-07-23 23:48:26-INFO-Reduction of feasible pairs by 99.47%
11-07-23 23:48:26-INFO-Degree 2 	Completed
11-07-23 23:48:26-INFO-trips to extend at degree 2 : 36
11-07-23 23:48:26-INFO-At degree 2 feasible extensions found out of 0 searched
11-07-23 23:48:26-INFO-Degree 3 	Completed
11-07-23 23:48:26-INFO-No more trips to exted at degree 3


In [274]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

11-07-23 23:48:26-INFO-Matching 50 trips to 86 rides in order to min u_veh
11-07-23 23:48:26-INFO-Problem solution: Optimal. 
Total costs for single trips:         14,866 
reduced by matching to:               13,371


In [275]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],6.219000,336,1,[6.219],"[0, 336]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44831869, 44734726]","[None, 0, 0]",node time req_id od 0 NaN...,336,3.36,5.043000,1.008600,4.0344
1,[1],1.826500,98,1,[1.8265],"[112, 98]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1833104239, 1448535819]","[None, 1, 1]",node time req_id od 0 ...,98,0.98,1.483500,0.296700,1.1868
2,[2],6.004500,324,1,[6.0045],"[274, 324]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44857299, 44820010]","[None, 2, 2]",node time req_id od 0 NaN...,324,3.24,4.870500,0.974100,3.8964
3,[3],6.815500,368,1,[6.8155],"[391, 368]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44894214, 1448535914]","[None, 3, 3]",node time req_id od 0 ...,368,3.68,5.527500,1.105500,4.4220
4,[4],4.188500,226,1,[4.1885],"[407, 226]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44837766, 1530303662]","[None, 4, 4]",node time req_id od 0 ...,226,2.26,3.397500,0.679500,2.7180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,"[29, 33]",7.347775,457,21,"[6.16695, 1.180825]","[2340.5, 255, 72, 130]","[29, 33]","[33, 29]",2,81,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1571087335, 1010679665, 4862057242, 144...","[None, 29, 33, 33, 29]",node time req_id od 0 ...,457,4.57,4.854375,0.970875,3.8835
82,"[36, 43]",10.149450,532,21,"[7.18275, 2.9667]","[2773.0, 283, 181, 68]","[36, 43]","[43, 36]",2,82,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44742832, 2321923801, 44871803, 1505631...","[None, 36, 43, 43, 36]",node time req_id od 0 ...,532,5.32,6.696000,1.339200,5.3568
83,"[11, 15]",8.771675,612,21,"[7.845025, 0.92665]","[842.5, 427, 57, 128]","[11, 15]","[15, 11]",2,83,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 5739211290, 1584502762, 1679761151, 160...","[None, 11, 15, 15, 11]",node time req_id od 0 ...,612,6.12,5.695875,1.139175,4.5567
84,"[8, 9]",8.644600,370,21,"[5.80115, 2.84345]","[527.0, 189, 166, 15]","[8, 9]","[9, 8]",2,84,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",1,"[None, 44768472, 4349049062, 44861089, 2635624...","[None, 8, 9, 9, 8]",node time req_id od 0 ...,370,3.70,5.769000,1.153800,4.6152


In [276]:
sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate

11-07-23 23:48:28-INFO-Matching 50 trips to 86 rides in order to min u_veh
11-07-23 23:48:28-INFO-Problem solution: Optimal. 
Total costs for single trips:         14,866 
reduced by matching to:               13,371
0.0005
11-07-23 23:48:30-CRITICAL-this is request 0 with [0] available rides.
11-07-23 23:48:30-CRITICAL-ride 0 available [1]
11-07-23 23:48:30-CRITICAL-this is reuqest 0 with [0] still available rides.
11-07-23 23:48:30-CRITICAL-vehicle 1 has 1 choices
0.0005
11-07-23 23:48:30-CRITICAL-this is request 1 with [1] available rides.
11-07-23 23:48:30-CRITICAL-ride 1 available [1]
11-07-23 23:48:30-CRITICAL-this is reuqest 1 with [1] still available rides.
11-07-23 23:48:30-CRITICAL-vehicle 2 has 1 choices
0.0005
11-07-23 23:48:30-CRITICAL-this is request 2 with [2, 52, 60, 72] available rides.
11-07-23 23:48:30-CRITICAL-ride 2 available [1]
11-07-23 23:48:30-CRITICAL-ride 52 available [1, 0]
11-07-23 23:48:30-CRITICAL-ride 60 available [1, 0]
11-07-23 23:48:30-CRITICAL-ride 7

In [277]:
sim.res[0].veh_exp

,nRIDES,nREJECTED,OUT,STARTS_DAY,OPENS_APP,RECEIVES_REQUEST,ACCEPTS_REQUEST,REJECTS_REQUEST,IS_ACCEPTED_BY_TRAVELLER,IS_REJECTED_BY_TRAVELLER,...,STARTS_REPOSITIONING,REPOSITIONED,DECIDES_NOT_TO_DRIVE,ENDS_SHIFT,TRAVEL,WAIT,CRUISE,OPERATIONS,IDLE,PAX_KM
1,3.0,0.0,False,0,0,1714.0,30.0,0,213.0,0,...,0,0,0,7670.0,744.0,0,338.0,333.0,6255.0,10.184
2,4.0,0.0,False,0,0,1514.0,45.0,0,149.0,0,...,0,0,0,8182.0,515.0,0,274.0,314.0,7079.0,8.385
3,1.0,0.0,False,0,0,3206.0,15.0,0,20.0,0,...,0,0,0,7369.0,89.0,0,70.0,65.0,7145.0,0.891
4,3.0,0.0,False,0,0,2040.0,45.0,0,60.0,0,...,0,0,0,7465.0,960.0,0,139.0,195.0,6171.0,9.620
5,2.0,0.0,False,0,0,2595.0,15.0,0,198.0,0,...,0,0,0,7110.0,329.0,0,492.0,273.0,6016.0,7.465
6,2.0,0.0,False,0,0,2916.0,15.0,0,150.0,0,...,0,0,0,6995.0,363.0,0,300.0,225.0,6107.0,8.467
7,2.0,0.0,False,0,0,1609.0,15.0,0,60.0,0,...,0,0,0,8382.0,464.0,0,209.0,135.0,7574.0,6.364
8,3.0,0.0,False,0,0,1867.0,26.0,0,44.0,0,...,0,0,0,7963.0,642.0,0,167.0,160.0,6994.0,8.836
9,5.0,0.0,False,0,0,1658.0,60.0,0,184.0,0,...,0,0,0,7073.0,1076.0,0,598.0,394.0,5005.0,14.500
10,4.0,0.0,False,0,0,2329.0,45.0,0,158.0,0,...,0,0,0,6725.0,1080.0,0,342.0,323.0,4980.0,13.862


In [287]:
sim.vehs[2].rdf

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,sim_schedule,ttrav,dist,fare,commission,driver_revenue,pickup_dist,trav_dist,operating_cost,profit
1,[1],1.8265,98,1,[1.8265],"[112, 98]",[1],[1],1,1,...,node time req_id od 0 ...,98,0.98,1.48350,0.29670,1.1868,674,674.98,0.337490,0.849310
59,"[8, 13]",9.8583,603,20,"[5.949025, 3.909275]","[630.5, 368, 27, 208]","[8, 13]","[8, 13]",2,59,...,node time req_id od 0 ...,603,6.03,6.48675,1.29735,5.1894,1470,1476.03,0.738015,4.451385
24,[24],4.1930,226,1,[4.1930000000000005],"[1895, 226]",[24],[24],1,24,...,node time req_id od 0 ...,226,2.26,3.40200,0.68040,2.7216,41,43.26,0.021630,2.699970
35,[35],5.9675,322,1,[5.967499999999999],"[2617, 322]",[35],[35],1,35,...,node time req_id od 0 NaN...,322,3.22,4.84050,0.96810,3.8724,622,625.22,0.312610,3.559790


In [288]:
pd.DataFrame(sim.vehs[2].myrides)

,veh,pos,t,event,paxes
0,2,1.510592e+09,0,STARTS_DAY,[]
1,2,1.510592e+09,0,OPENS_APP,[]
2,2,1.510592e+09,112,RECEIVES_REQUEST,[]
3,2,1.510592e+09,127,ACCEPTS_REQUEST,[]
4,2,1.510592e+09,147,IS_ACCEPTED_BY_TRAVELLER,[None]
5,2,1.833104e+09,194,ARRIVES_AT_PICKUP,[]
6,2,1.833104e+09,194,MEETS_TRAVELLER_AT_PICKUP,[]
7,2,1.833104e+09,224,DEPARTS_FROM_PICKUP,[1]
8,2,1.448536e+09,322,ARRIVES_AT_DROPOFF,[1]
9,2,1.448536e+09,575,RECEIVES_REQUEST,[]
